In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

In [ ]:
data_path = os.path.join("..", "data")

dir_store_path = os.path.join(data_path, "temporary results")

full_dataset_path = os.path.join(dir_store_path, "full_dataset.xlsx")

In [ ]:
dataset_df = pd.read_excel(full_dataset_path)

In [ ]:
dataset_df

### Get TNTC samples

In [ ]:
tntc_df = dataset_df[
    (dataset_df["Coliform (1ml)"] == "TNTC")
    | (dataset_df["Ecoli (1ml)"] == "TNTC")
    | (dataset_df["Coliform (1ml)"] == 0)
]

In [ ]:
tntc_df

In [ ]:
clean_df = dataset_df.drop(tntc_df.index)

## TNTC analysis

### Get measurements that have at least one TNTC

In [ ]:
related_tntc_df = pd.merge(
    left=clean_df,
    right=tntc_df,
    on=["DateTime", "Site", "Bottle"],
    how="inner",
)

# Get columns that end with '_y'
cols_to_drop = related_tntc_df.filter(regex="_y$").columns

# Drop these columns
related_tntc_df = related_tntc_df.drop(columns=cols_to_drop)

related_tntc_df.rename(
    columns={
        "Sample_x": "Sample",
        "Image Date Time_x": "Image Date Time",
        "Dilution_x": "Dilution",
        "Coliform (1ml)_x": "Coliform (1ml)",
        "Ecoli (1ml)_x": "Ecoli (1ml)",
        "Technician Counting_x": "Technician Counting",
        "Technician Water Quality_x": "Technician Water Quality",
        "Temp C_x": "Temp C",
        "Ph_x": "Ph",
        "Cond (ms)_x": "Cond (ms)",
    },
    inplace=True,
)

related_tntc_df = pd.concat([related_tntc_df, tntc_df])

related_tntc_df = related_tntc_df.sort_values(by=["DateTime", "Site", "Bottle"])

related_tntc_df.drop_duplicates(inplace=True)

In [ ]:
related_tntc_df

In [ ]:
clean_df["Coliform (1ml)"] = clean_df["Coliform (1ml)"].astype("float64")
clean_df["Ecoli (1ml)"] = clean_df["Ecoli (1ml)"].astype("float64")

In [ ]:
clean_df

In [ ]:
clean_df = (
    clean_df.groupby(["DateTime", "Site", "Bottle", "Sample"], as_index=False)
    .agg(
        {
            "Technician Water Quality": "first",
            "Technician Counting": "first",
            "Temp C": ["mean", "std"],
            "Ph": ["mean", "std"],
            "Cond (ms)": ["mean", "std"],
            "Coliform (1ml)": ["mean", "std"],
            "Ecoli (1ml)": ["mean", "std"],
        },
    )
    .reset_index()
)

In [ ]:
clean_df = clean_df[
    clean_df[["Coliform (1ml)", "Ecoli (1ml)", "Temp C", "Ph", "Cond (ms)"]]
    .notnull()
    .all(axis=1)
]

In [ ]:
clean_df.columns = [
    "_".join(col) if col[1] == "mean" or col[1] == "std" else col[0]
    for col in clean_df.columns.values
]

In [ ]:
clean_df

In [ ]:
%%script false --no-raise-error
clean_df["CV_Coliform"] = (
    clean_df["Coliform (1ml)_std"] / clean_df["Coliform (1ml)_mean"]
)
clean_df["CV_Ecoli"] = (
    clean_df["Ecoli (1ml)_std"] / clean_df["Ecoli (1ml)_mean"]
)

# Per Site Data Visualization and Analysis

## General Info

In [ ]:
site_dict = {}
for site in clean_df["Site"].unique():
    site_dict[site] = clean_df[clean_df["Site"] == site]

In [ ]:
for site in site_dict:
    print("Site: ", site)
    print("-" * 30)
    print(site_dict[site].describe().to_string())
    print("\n")
    print(
        "Timespan: "
        + pd.to_datetime(site_dict[site]["DateTime"])
        .min()
        .strftime("%Y-%m-%d %H:%M:%S")
        + " - "
        + pd.to_datetime(site_dict[site]["DateTime"])
        .max()
        .strftime("%Y-%m-%d %H:%M:%S")
    )
    print("\n")

## Hypothesis Tests Correlations

### Coliform

In [ ]:
for site in site_dict:
    print("-" * 30)
    print("Site: ", site)
    r, p = stats.pearsonr(
        site_dict[site]["Temp C_mean"], site_dict[site]["Coliform (1ml)_mean"]
    )

    print("Pearsons correlation: ", r)
    print("Pearsons p-value: ", p)

In [ ]:
for site in site_dict:
    print("-" * 30)
    print("Site: ", site)
    r, p = stats.pearsonr(
        site_dict[site]["Ph_mean"], site_dict[site]["Coliform (1ml)_mean"]
    )

    print("Pearsons correlation: ", r)
    print("Pearsons p-value: ", p)

In [ ]:
for site in site_dict:
    print("-" * 30)
    print("Site: ", site)
    r, p = stats.pearsonr(
        site_dict[site]["Cond (ms)_mean"],
        site_dict[site]["Coliform (1ml)_mean"],
    )

    print("Pearsons correlation: ", r)
    print("Pearsons p-value: ", p)

### Ecoli

In [ ]:
for site in site_dict:
    print("-" * 30)
    print("Site: ", site)
    r, p = stats.pearsonr(
        site_dict[site]["Temp C_mean"], site_dict[site]["Ecoli (1ml)_mean"]
    )

    print("Pearsons correlation: ", r)
    print("Pearsons p-value: ", p)

In [ ]:
for site in site_dict:
    print("-" * 30)
    print("Site: ", site)
    r, p = stats.pearsonr(
        site_dict[site]["Ph_mean"], site_dict[site]["Ecoli (1ml)_mean"]
    )

    print("Pearsons correlation: ", r)
    print("Pearsons p-value: ", p)

In [ ]:
for site in site_dict:
    print("-" * 30)
    print("Site: ", site)
    r, p = stats.pearsonr(
        site_dict[site]["Cond (ms)_mean"], site_dict[site]["Ecoli (1ml)_mean"]
    )

    print("Pearsons correlation: ", r)
    print("Pearsons p-value: ", p)

In [ ]:
for site in site_dict:
    print("-" * 30)
    print("Site: ", site)
    r, p = stats.pearsonr(
        site_dict[site]["Ecoli (1ml)_mean"],
        site_dict[site]["Coliform (1ml)_mean"],
    )

    print("Pearsons correlation: ", r)
    print("Pearsons p-value: ", p)

## Correlation Matrix Heatmap

In [ ]:
cols = [
    "Temp C_mean",
    "Ph_mean",
    "Cond (ms)_mean",
    "Coliform (1ml)_mean",
    "Ecoli (1ml)_mean",
]


# Pearson, used for two quantitative continuous variables which have a linear relationship
# Spearman, used for two quantitative variables if the link is partially linear, or for one qualitative ordinal variable and one quantitative variable
# Kendall, often used for two qualitative ordinal variables

for site in site_dict:
    corr = site_dict[site][cols].corr(method="pearson")

    plt.figure(figsize=(5, 5))
    plt.title("Site: " + site)
    ax = sns.heatmap(
        corr,
        vmin=-1,
        vmax=1,
        center=0,
        cmap=sns.diverging_palette(20, 220, n=200),
        square=True,
        annot=True,
        fmt=".3f",
    )
    ax.set_xticklabels(
        ax.get_xticklabels(), rotation=45, horizontalalignment="right"
    )

## Scatter Plots

In [ ]:
cols = [
    "Temp C_mean",
    "Ph_mean",
    "Cond (ms)_mean",
    "Coliform (1ml)_mean",
    "Ecoli (1ml)_mean",
]

for site in site_dict:
    plot = sns.pairplot(data=site_dict[site][cols])
    plot.fig.suptitle("Site: " + site, y=1.08)

## Boxplots

In [ ]:
cols = [
    "Temp C_mean",
    "Ph_mean",
    "Cond (ms)_mean",
    "Coliform (1ml)_mean",
    "Ecoli (1ml)_mean",
]

for site in site_dict:
    for col in cols:
        sns.boxplot(y=site_dict[site][col], orient="v")
        plt.title("Site: " + site + " - " + col)
        plt.show()

## Timeseries

In [ ]:
for site in site_dict:
    for col in cols:
        site_dict[site].plot(
            x="DateTime", y=col, figsize=(15, 5), grid=True, kind="scatter"
        )
        plt.title("Site: " + site + " - " + col)

## Check Input Distribution Difference TNTC/noTNTC

In [ ]:
for site in clean_df["Site"].unique():
    clean = clean_df[clean_df["Site"] == site]
    tntc = tntc_df[tntc_df["Site"] == site]

    fig, axs = plt.subplots(3, figsize=(15, 15))
    features = ["Temp C_mean", "Ph_mean", "Cond (ms)_mean"]
    tntc_features = ["Temp C", "Ph", "Cond (ms)"]

    for i, ax in enumerate(axs):
        ax.hist(
            clean[features[i]], color="blue", alpha=0.5, bins=30, label="clean"
        )
        ax.hist(
            tntc[tntc_features[i]],
            color="red",
            alpha=0.5,
            bins=30,
            label="tntc",
        )
        ax.set_title(features[i])
        ax.legend(loc="upper right")

    plt.suptitle("Site: " + site)
    plt.tight_layout()
    plt.show()

# Overall Data Visualization and Analysis

## General Info

In [ ]:
overall_df = clean_df.copy()

In [ ]:
overall_df.drop(columns=["Site"], inplace=True)

In [ ]:
print(overall_df.describe().to_string())
print("\n")
print(
    "Timespan: "
    + pd.to_datetime(overall_df["DateTime"]).min().strftime("%Y-%m-%d %H:%M:%S")
    + " - "
    + pd.to_datetime(overall_df["DateTime"]).max().strftime("%Y-%m-%d %H:%M:%S")
)

## Hypothesis Tests Correlations

### Coliform

In [ ]:
r, p = stats.pearsonr(
    overall_df["Temp C_mean"], overall_df["Coliform (1ml)_mean"]
)

print("Pearsons correlation: ", r)
print("Pearsons p-value: ", p)

In [ ]:
r, p = stats.pearsonr(overall_df["Ph_mean"], overall_df["Coliform (1ml)_mean"])

print("Pearsons correlation: ", r)
print("Pearsons p-value: ", p)

In [ ]:
r, p = stats.pearsonr(
    overall_df["Cond (ms)_mean"], overall_df["Coliform (1ml)_mean"]
)

print("Pearsons correlation: ", r)
print("Pearsons p-value: ", p)

### Ecoli

In [ ]:
r, p = stats.pearsonr(overall_df["Temp C_mean"], overall_df["Ecoli (1ml)_mean"])

print("Pearsons correlation: ", r)
print("Pearsons p-value: ", p)

In [ ]:
r, p = stats.pearsonr(overall_df["Ph_mean"], overall_df["Ecoli (1ml)_mean"])

print("Pearsons correlation: ", r)
print("Pearsons p-value: ", p)

In [ ]:
r, p = stats.pearsonr(
    overall_df["Cond (ms)_mean"], overall_df["Ecoli (1ml)_mean"]
)

print("Pearsons correlation: ", r)
print("Pearsons p-value: ", p)

In [ ]:
r, p = stats.pearsonr(
    overall_df["Coliform (1ml)_mean"], overall_df["Ecoli (1ml)_mean"]
)

print("Pearsons correlation: ", r)
print("Pearsons p-value: ", p)

## Correlation Matrix Heatmap

In [ ]:
cols = [
    "Temp C_mean",
    "Ph_mean",
    "Cond (ms)_mean",
    "Coliform (1ml)_mean",
    "Ecoli (1ml)_mean",
]


# Pearson, used for two quantitative continuous variables which have a linear relationship
# Spearman, used for two quantitative variables if the link is partially linear, or for one qualitative ordinal variable and one quantitative variable
# Kendall, often used for two qualitative ordinal variables

corr = overall_df[cols].corr(method="pearson")

plt.figure(figsize=(5, 5))
ax = sns.heatmap(
    corr,
    vmin=-1,
    vmax=1,
    center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True,
    annot=True,
    fmt=".3f",
)
ax.set_xticklabels(
    ax.get_xticklabels(), rotation=45, horizontalalignment="right"
)

## Scatter Plots

In [ ]:
cols = [
    "Temp C_mean",
    "Ph_mean",
    "Cond (ms)_mean",
    "Coliform (1ml)_mean",
    "Ecoli (1ml)_mean",
]


plot = sns.pairplot(data=overall_df[cols])

## Boxplots

In [ ]:
cols = [
    "Temp C_mean",
    "Ph_mean",
    "Cond (ms)_mean",
    "Coliform (1ml)_mean",
    "Ecoli (1ml)_mean",
]

for col in cols:
    sns.boxplot(y=overall_df[col], orient="v")
    plt.title(col)
    plt.show()

## Timeseries

In [ ]:
cols = [
    "Temp C_mean",
    "Ph_mean",
    "Cond (ms)_mean",
    "Coliform (1ml)_mean",
    "Ecoli (1ml)_mean",
]

for col in cols:
    overall_df.plot(x="DateTime", y=col, figsize=(15, 5), grid=True)
    plt.title(col)

## Check Input Distribution Difference TNTC/noTNTC

In [ ]:
tntc = tntc_df.drop(columns=["Site"])

fig, axs = plt.subplots(3, figsize=(15, 15))
features = ["Temp C_mean", "Ph_mean", "Cond (ms)_mean"]
tntc_features = ["Temp C", "Ph", "Cond (ms)"]

for i, ax in enumerate(axs):
    ax.hist(
        overall_df[features[i]], color="blue", alpha=0.5, bins=30, label="clean"
    )
    ax.hist(
        tntc[tntc_features[i]],
        color="red",
        alpha=0.5,
        bins=30,
        label="tntc",
    )
    ax.set_title(features[i])
    ax.legend(loc="upper right")

plt.tight_layout()
plt.show()

# Further Processing

In [ ]:
clean = clean_df.copy()
tntc = tntc_df.copy()

In [ ]:
rename_dict = {
    "Temp C": "Temp C_mean",
    "Ph": "Ph_mean",
    "Cond (ms)": "Cond (ms)_mean",
    "Coliform (1ml)": "Coliform (1ml)_mean",
    "Ecoli (1ml)": "Ecoli (1ml)_mean",
}

In [ ]:
tntc.rename(columns=rename_dict, inplace=True)

In [ ]:
tntc.drop(columns=["Image Date Time", "Dilution"], inplace=True)

In [ ]:
clean_df

In [ ]:
clean_df.to_excel(
    os.path.join(dir_store_path, "clean_dataset.xlsx"), index=False
)

In [ ]:
tntc_df.to_excel(os.path.join(dir_store_path, "tntc_dataset.xlsx"), index=False)

In [ ]:
full_df = pd.concat([clean, tntc])

### Processed clean_df with means and stds computed + TNTC

In [ ]:
full_df

In [ ]:
full_df.drop(
    columns=[
        "Temp C_std",
        "Ph_std",
        "Cond (ms)_std",
        "Coliform (1ml)_std",
        "Ecoli (1ml)_std",
    ],
    inplace=True,
)

In [ ]:
processed_dataset_path = os.path.join(dir_store_path, "processed_dataset.xlsx")

full_df.to_excel(processed_dataset_path)